# Big Data Modeling and Management Assigment - Homework 1

# Submission

GROUP NUMBER: **12** - please add your group number into the file name

GROUP MEMBERS:

|STUDENT NAME|STUDENT NUMBER|
|---|---|
|Catarina Gonçalves Nunes|20230083|
|Sofia Jacinto|20240598|
|José Cavaco|20240513|
|José Tiago|20240582|

## 🍺 The Beer project  🍺 

As it was shown in classes, graph databases are a natural way of navegating related information. For this first project we will be taking a graph database to analyse beer and breweries!   

The project datasets are based on [kaggle](https://www.kaggle.com/ehallmar/beers-breweries-and-beer-reviews), released by Evan Hallmark. 

### Problem description

Imagine you are working in the Data Management department of Analytics company.
Explore the database via python neo4j connector and/or the graphical tool in the NEO4J webpage. Answer the questions while adjusting the database to meet the needs of your colleagues.
Please record and keep track of your database changes, and submit the file with all cells run and with the output shown.

### Questions

1. Explore the database: get familiar with current schema, elements and other important database parameters. [1 point]
2. Adjust the database and mention reasoning behind: e.g. clean errors, remove redundancies, adjust schema as necessary. Visualize the final version of database schema. [4 points]
3. Analytics department requires the following information for the biweekly reporting: [5 points]
    1. How many reviews has the beer with the most reviews?
    2. Which three users wrote the most reviews about beers?
    3. Find all beers that are described with following words: 'fruit', 'complex', 'nutty', 'dark'.
    4. Which top three breweries produce the largest variety of beer styles?
    5. Which country produces the most beer styles?
4. Market Analysis department in your company accesses and updates the trends data on the daily basis. Given that, consider how you need to optimize the database and its performance so that the following queries are efficient. Measure performance to communicate your improvements using PROFILE before final query. Answer the following: [4 points]
    1. Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery? Keep in mind that the strongest known beer is Snake Venom, and deal with the error entries in the database.
    2. Using the answer from question 2, find the top 5 distict beer styles with the highest average score of smell + feel that were reviewed by the third most productive user. Keep in mind that cleaning the database earlier should ensure correct results.
5. Answer **two out of four** of the following questions using Graph Algorithms (gds): [NB: make sure to clear the graph before using it again] For the quarterly report, Analytics department the follownig information. [6 points]
    1. Which two countries are most similiar when it comes to their top five most produced Beer styles?
    2. Which beer is the most popular when considering the number of users who reviewed it? 
    3. Users are connected together by their reviews of beers, taking into consideration the "smell" score they assign as a weight, how many communities are formed from these relationships? How many users are in the three largest communities? 
    4. Which user is the most influential when it comes to reviews of distinct beers by style?
 
### Groups  

Groups should have 4 people maximum. Please mark which group you are here: https://shorturl.at/zE0QP 

### Submission      

The code used to produce the results and to-the-point explations should be uploaded to moodle. They should have a clear reference to the group, either on the file name or on the document itself. Preferably one Jupyter notebook per group.

Delivery date: Until the **midnight of March 18, 2025**

### Evaluation   

This will be 20% of the final grade.   
Each solution will be evaluated on 2 components: correctness of results and efficiency of the query (based on database schema).  
All code will go through plagiarism automated checks. Groups with the same code will undergo investigation.

## Loading the Database

#### Be sure that you **don't have** the neo4j docker container from the classes running (you can Stop it in the desktop app or with the command "`docker stop Neo4JLab`")


The default container does not have any data whatsoever, we will have to load a database into our docker image:
- Download and unzip the `Neo4JHWData` file provided in Moodle.
- Copy the path of the `Neo4JHWData` folder of the unziped file, e.g. `C:/PATH/Neo4JHWData/data`.
- Download and unzip the `Neo4JPlugins` file provided in Moodle.
- Copy the path of the `Neo4JPlugins` folder of the unziped file, e.g. `C:/PATH/Neo4Jplugins`.
- Change the code below accordingly. As you might have noticed, you do not have a user called `nunoa`, please use the appropriate path that you got from the previous step. Be sure that you have a neo4j docker container running: \

`docker run --name Neo4JHW2025 -p 7474:7474 -p 7687:7687 -d -v "c:\PATH\Neo4JPlugins":/plugins -v "c:\PATH\Neo4JHWData\data":/data --env NEO4J_AUTH=neo4j/test --env NEO4J_dbms_connector_https_advertised__address="localhost:7473" --env NEO4J_dbms_connector_http_advertised__address="localhost:7474" --env NEO4J_dbms_connector_bolt_advertised__address="localhost:7687" --env NEO4J_dbms_security_procedures_unrestricted=gds.* --env NEO4J_dbms_security_procedures_allowlist="gds.*" neo4j:4.4.5`

- Since Neo4j is trying to recognize a new database folder, this might take a bit (let's say 3 minutes), so don't worry.

If the neo4j browser fails to load gds plugins, run the following in the Command Prompt before creating the container again:
`// Remove stopped containers //
docker container prune -f
// Remove unused images //
docker image prune -a -f
// Remove unused volumes //
docker volume prune -f
// Remove unused networks //
docker network prune -f
// Remove all unused resources in one command //
docker system prune -a -f`

`Our Notes`:

To handle large-scale operations, such as modifying relationships on a high volume of data (like reviews), we had to install the APOC (Awesome Procedures on Cypher) library. Specifically, we added apoc-4.4.0.9-all.jar to the Plugins folder, as it was the only way to efficiently execute batch operations in an iterative manner. Without it, handling large data transformations, such as reversing relationship directions, would not have been feasible.

We used the following Docker command to run Neo4j with APOC enabled and mounted the plugins directory:

docker run --hostname=9ad48e239b09 \
  --env=NEO4J_dbms_connector_bolt_advertised__address=localhost:7687 \
  --env=NEO4J_dbms_connector_http_advertised__address=localhost:7474 \
  --env=NEO4J_dbms_security_procedures_unrestricted=apoc.*,gds.* \
  --env=NEO4J_dbms_security_procedures_allowlist=apoc.*,gds.* \
  --env=NEO4J_AUTH=neo4j/test \
  --volume=C:\Users\xxxx\Desktop\big-data-modeling\Neo4JPlugins\Neo4JPlugins:/plugins \
  --volume=C:\Users\xxxx\Desktop\big-data-modeling\Neo4JHWData\NEW_VERSION\data:/data \
  -p 7474:7474 -p 7687:7687 \
  --restart=no -d neo4j:4.4.5

## Index 

<a id="index">

1. [Explore the Database: Understanding the Current Schema, Elements, and Important Parameters](#explore-the-database)
2. [Adjust the Database and Mention the Reasoning Behind the Adjustments](#adjust-the-database) 
3. [Analytics Department Biweekly Reporting Requirements](#analytics-department)
   1. [How many reviews does the beer with the most reviews have?](#beer-with-most-reviews)
   2. [Which three users wrote the most reviews about beers?](#three-users-most-reviews)
   3. [Find all beers that are described with the following words: 'fruit', 'complex', 'nutty', 'dark'.](#beers-with-descriptions)
   4. [Which top three breweries produce the largest variety of beer styles?](#top-three-breweries)
   5. [Which country produces the most beer styles?](#country-most-styles)
4. [Optimizing Database Performance for Market Analysis Department](#optimizing-performance)
   1. [Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery.](#strongest-beers)
   2. [Find the top 5 distinct beer styles with the highest average score of smell and feel that were reviewed by the third most productive user.](#top-5-beer-styles)
5. [Answer Two Out of Four Questions Using Graph Algorithms (gds)](##graph-algorithms)
   1. [Which two countries are most similar when it comes to their top five most produced Beer styles?](#similar-countries)
   2. [Which beer is the most popular considering the number of users who reviewed it?](#most-popular-beer)
   3. [How many communities are formed from these relationships? How many users are in the three largest communities?](#user-communities)
   4. [Which user is the most influential when it comes to reviews of distinct beers by style?](#influential-user)

### <div class="alert-warning" style="background-color: #FFF9C4; color: black; padding: 10px;"> <a id="explore-the-database"></a>1. Explore the Database: Understanding the Current Schema, Elements, and Important Parameters </div>

In [52]:
#! pip install neo4j

In [53]:
from neo4j import GraphDatabase
from pprint import pprint

In [54]:
NEO4J_URI="neo4j://localhost:7687"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="test"

In [55]:
driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD), )

In [56]:
def execute_read(driver, query):    
    with driver.session(database="neo4j") as session:
        result = session.execute_read(lambda tx, query: list(tx.run(query)), query)
    return result

In [57]:
def execute_write(driver, query):
    with driver.session(database="neo4j") as session:
        # Write transactions allow the driver to handle retries and transient errors
        result = session.execute_write(lambda tx, query: list(tx.run(query)), query)
    return result

`Note:` We ill overwrite the variable "result" in almost all our answers to save memory bc we don't need to reuse these results.

#### 1.1 Dataset nodes (labels), Relationships and Schema:

In [58]:
query = """
       call db.labels();
    """

result = execute_read(driver, query)

pprint(result)

[<Record label='COUNTRIES'>,
 <Record label='CITIES'>,
 <Record label='BREWERIES'>,
 <Record label='BEERS'>,
 <Record label='REVIEWS'>,
 <Record label='STYLE'>,
 <Record label='USER'>]


make the names consistent

In [59]:
# Count all nodes
query = """ 
    MATCH (n) RETURN count(n) as count """
result = execute_read(driver, query)
pprint(result)

[<Record count=3215489>]


In [60]:
query = """
        CALL db.relationshipTypes();
    """
result = execute_read(driver, query)

pprint(result)

[<Record relationshipType='REVIEWED'>,
 <Record relationshipType='BREWED'>,
 <Record relationshipType='IN'>,
 <Record relationshipType='HAS_STYLE'>,
 <Record relationshipType='POSTED'>]


In [61]:
query = """
        // Count all relationships
    MATCH ()-->() RETURN count(*);
    """
result = execute_read(driver, query)

pprint(result)

[<Record count(*)=5856205>]


In [62]:
query = """
    CALL db.schema.visualization();
"""
result = execute_read(driver, query)
pprint(result)

[<Record nodes=[<Node element_id='-5' labels=frozenset({'REVIEWS'}) properties={'name': 'REVIEWS', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-4' labels=frozenset({'BEERS'}) properties={'name': 'BEERS', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-1' labels=frozenset({'COUNTRIES'}) properties={'name': 'COUNTRIES', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-3' labels=frozenset({'BREWERIES'}) properties={'name': 'BREWERIES', 'indexes': ['id'], 'constraints': []}>, <Node element_id='-6' labels=frozenset({'STYLE'}) properties={'name': 'STYLE', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-2' labels=frozenset({'CITIES'}) properties={'name': 'CITIES', 'indexes': ['name'], 'constraints': []}>, <Node element_id='-7' labels=frozenset({'USER'}) properties={'name': 'USER', 'indexes': ['name'], 'constraints': []}>] relationships=[<Relationship element_id='-1' nodes=(<Node element_id='-4' labels=frozenset({'BEERS'}) properties={'name': '

From the schema visualization, the database contains the following nodes and relationships:

- CITIES → Connected to COUNTRIES (relationship: IN), indicating that a city belongs to a specific country. It is also connected to itself with an IN relationship, which seems unusual and should be reviewed.
- BREWERIES → Connected to BEERS (relationship: BREWED), meaning that a brewery produces multiple beers. It is also connected to CITIES and COUNTRIES (both relationships: IN), indicating the location of the brewery at both the city and country levels.
- BEERS → Connected to STYLE (relationship: HAS_STYLE), specifying the beer’s style, and to REVIEWS (relationship: REVIEWED), linking beers to user reviews.
- REVIEWS → Connected to USER (relationship: POSTED) - **seems inverted**

![Database Schema Visualization](https://raw.githubusercontent.com/CatarinaGN/Big-Data-Modelling-and-Management/main/db.schema.visualization%28%29.png)


Observations for adjustments:
- BREWERIES connected to both CITIES and COUNTRIES → This could be redundant if cities already belong to countries. If every brewery has a city assigned, you can infer the country, making the direct IN relationship between breweries and countries unnecessary.
- CITIES connected to itself (IN relationship) is unusual and might be an error. We should check if this represents hierarchical regions (e.g., neighborhoods within cities) or if it's a mistake
- Check the direction of the following relationships: (REVIEW → POSTED → User) should be (:USER) -[:POSTED]-> (:REVIEWS), and  (BEER → REVIEWED → REVIEW) should be (:BEERS) -[:REVIEWED_BY]-> (:REVIEWS):

In [63]:
query = """
    MATCH (c1:CITIES)-[r:IN]->(c2:CITIES)
    RETURN c1.name AS city, c2.name AS parent_city, COUNT(*) AS count
    ORDER BY count DESC
"""
result = execute_read(driver, query)
pprint(result)

[]


- There are no cities linked to other cities using the IN relationship ➡ This suggests that cities are not hierarchical (i.e., no neighborhoods inside cities).

In [64]:
query = """
    MATCH (a)-[r]->(b)
    RETURN DISTINCT labels(a) AS StartNode, type(r) AS Relationship, labels(b) AS EndNode, COUNT(*) AS count
    ORDER BY count DESC;
"""
result = execute_read(driver, query)
pprint(result)

[<Record StartNode=['REVIEWS'] Relationship='POSTED' EndNode=['USER'] count=2538044>,
 <Record StartNode=['BEERS'] Relationship='REVIEWED' EndNode=['REVIEWS'] count=2537991>,
 <Record StartNode=['BREWERIES'] Relationship='BREWED' EndNode=['BEERS'] count=358873>,
 <Record StartNode=['BEERS'] Relationship='HAS_STYLE' EndNode=['STYLE'] count=358873>,
 <Record StartNode=['BREWERIES'] Relationship='IN' EndNode=['CITIES'] count=50347>,
 <Record StartNode=['CITIES'] Relationship='IN' EndNode=['COUNTRIES'] count=12077>]


so yes, the relations are inverted and another problem is:
- since we  have 2538044 POSTED relationships but only 2537991 REVIEWED relationships - That means 53 reviews are not linked to a beer (2538044 - 2537991 = 53) - we likely have orphaned reviews.

In [65]:
query = """
    MATCH (n)
    WHERE NOT EXISTS( (n)-[]-() )
    RETURN labels(n) AS Label, COUNT(n) AS OrphanedCount
"""
result = execute_read(driver, query)
pprint(result)

[<Record Label=['USER'] OrphanedCount=17290>,
 <Record Label=['COUNTRIES'] OrphanedCount=200>,
 <Record Label=['CITIES'] OrphanedCount=11665>,
 <Record Label=['BREWERIES'] OrphanedCount=50347>,
 <Record Label=['BEERS'] OrphanedCount=58873>,
 <Record Label=['REVIEWS'] OrphanedCount=11208>]


In [66]:
#nodes without labels
query = """
    MATCH (n)  
    WHERE size(labels(n)) = 0  
    RETURN count(n) AS nodes_without_labels;
"""
result = execute_read(driver, query)
pprint(result)

[<Record nodes_without_labels=0>]


#### 1.2 Duplicate entries:

In [67]:
query = """
       MATCH (n)
       RETURN DISTINCT labels(n) AS EntityType, keys(n) AS Properties;

    """

result = execute_read(driver, query)

pprint(result)

[<Record EntityType=['COUNTRIES'] Properties=['name']>,
 <Record EntityType=['CITIES'] Properties=['name']>,
 <Record EntityType=['BREWERIES'] Properties=['notes', 'types', 'id', 'name', 'state']>,
 <Record EntityType=['BREWERIES'] Properties=['notes', 'id', 'types', 'state', 'name']>,
 <Record EntityType=['BREWERIES'] Properties=['state', 'id', 'name', 'types', 'notes']>,
 <Record EntityType=['BREWERIES'] Properties=['notes', 'types', 'name', 'state', 'id']>,
 <Record EntityType=['BREWERIES'] Properties=['id', 'notes', 'state', 'types', 'name']>,
 <Record EntityType=['BREWERIES'] Properties=['id', 'notes', 'state', 'name', 'types']>,
 <Record EntityType=['BREWERIES'] Properties=['state', 'id', 'types', 'notes', 'name']>,
 <Record EntityType=['BREWERIES'] Properties=['notes', 'types', 'state', 'id', 'name']>,
 <Record EntityType=['BREWERIES'] Properties=['notes', 'id', 'name', 'types', 'state']>,
 <Record EntityType=['BREWERIES'] Properties=['id', 'types', 'state', 'name', 'notes']>,
 

- EntityType=['COUNTRIES'] Properties=['name'] ➡ we can check name for duplicates
- EntityType=['CITIES'] Properties=['name'] ➡ we can check name for duplicates
- EntityType=['STYLE'] Properties=['name'] ➡ we can check name for duplicates
- EntityType=['USER'] Properties=['name'] ➡ we can check name for duplicates
- Inconsistency in ['BREWERIES'] properties ordering (e.g., ['notes', 'types', 'id', 'name', 'state'] appears in multiple variations -> Properties should be consistent in ordering, but this is mostly a display issue rather than a functional problem. Same for ['BEERS'] (['id', 'abv', 'name', 'state', 'availability', 'retired', 'brewery_id', 'notes']) ➡ we can check name for duplicates
- and ['REVIEWS'] (['smell', 'look', 'beer_id', 'text', 'score', 'taste', 'feel', 'overall', 'id', 'date']) ➡ we need to check text for duplicates

In [68]:
query = """
    MATCH (n)
    WITH 
        labels(n)[0] AS entity, 
        COUNT(n) AS total, 
        COUNT(
            DISTINCT 
            CASE 
                WHEN 'name' IN keys(n) THEN n.name 
                WHEN 'text' IN keys(n) THEN n.text  // Use text for REVIEWS
                ELSE NULL 
            END
        ) AS unique
    RETURN entity, total, unique
    ORDER BY total DESC;


"""
result = execute_read(driver, query)
pprint(result)

[<Record entity='REVIEWS' total=2549252 unique=814333>,
 <Record entity='BEERS' total=417746 unique=298567>,
 <Record entity='USER' total=123935 unique=123935>,
 <Record entity='BREWERIES' total=100694 unique=45245>,
 <Record entity='CITIES' total=23330 unique=11665>,
 <Record entity='COUNTRIES' total=400 unique=200>,
 <Record entity='STYLE' total=113 unique=113>]


- ['REVIEWS'] ➡ check for duplicates;
- ['BEERS'] ➡ check for duplicates;
- ['USER'] ➡ all good;
- ['BREWERIES'] ➡ check for duplicates;
- ['CITIES'] ➡ check for duplicates;
- ['COUNTRIES'] ➡ check for duplicates;
- ['STYLE'] ➡ all good.

> Reviews analysis

In [69]:
query = """
    MATCH (r:REVIEWS)
    RETURN DISTINCT keys(r) AS properties
    LIMIT 10;
"""
result = execute_read(driver, query)
pprint(result)

[<Record properties=['text', 'smell', 'look', 'taste', 'feel', 'overall', 'beer_id', 'id', 'date', 'score']>,
 <Record properties=['look', 'text', 'smell', 'taste', 'feel', 'overall', 'beer_id', 'id', 'date', 'score']>,
 <Record properties=['smell', 'look', 'text', 'taste', 'feel', 'overall', 'beer_id', 'id', 'date', 'score']>,
 <Record properties=[]>,
 <Record properties=['beer_id', 'text', 'smell', 'look', 'score', 'taste', 'feel', 'overall', 'id', 'date']>,
 <Record properties=['look', 'beer_id', 'text', 'smell', 'score', 'taste', 'feel', 'overall', 'id', 'date']>,
 <Record properties=['text', 'beer_id', 'smell', 'look', 'score', 'taste', 'feel', 'overall', 'id', 'date']>,
 <Record properties=['smell', 'look', 'beer_id', 'text', 'score', 'taste', 'feel', 'overall', 'id', 'date']>]


first, we can see that some reviews might be empty.

In [70]:
query = """
    MATCH (r:REVIEWS)
    WHERE 
        r.text IS NULL OR trim(r.text) = "" OR
        r.smell IS NULL OR 
        r.look IS NULL OR 
        r.taste IS NULL OR 
        r.feel IS NULL OR 
        r.overall IS NULL OR 
        r.beer_id IS NULL OR 
        r.id IS NULL OR 
        r.date IS NULL OR 
        r.score IS NULL
    RETURN COUNT(*) AS total_empty_reviews;

"""
result = execute_read(driver, query)
pprint(result)

[<Record total_empty_reviews=19>]


19 identities are empty (have no properties), and can be removed

In [71]:
query = """
    MATCH (r:REVIEWS)
    RETURN COUNT(DISTINCT r.id) AS unique_review_ids;
"""
result = execute_read(driver, query)
pprint(result)

[<Record unique_review_ids=2546141>]


In [72]:
query = """
    MATCH (r:REVIEWS)
    RETURN COUNT(DISTINCT r.text) AS unique_review_texts;
"""
result = execute_read(driver, query)
pprint(result)

[<Record unique_review_texts=814333>]


we have 2 549 252 total reviews, 2 546 141 unique_review_ids and 814333 unique text from reviews. Since we have millions of reviews, analyzing duplicates in the same way as for other labels (such as BREWERIES or BEERS) is extremely computationally expensive, but we can try to explore this a bit further.

In [73]:
query = """
    MATCH (r:REVIEWS)
    WITH r.id AS review_id, COUNT(*) AS occurrences
    WHERE occurrences > 1
    RETURN review_id, occurrences
    ORDER BY occurrences DESC
    limit 10;
"""
result = execute_read(driver, query)
pprint(result)

[<Record review_id=None occurrences=19>,
 <Record review_id='28' occurrences=2>,
 <Record review_id='90' occurrences=2>,
 <Record review_id='45' occurrences=2>,
 <Record review_id='60' occurrences=2>,
 <Record review_id='86' occurrences=2>,
 <Record review_id='32' occurrences=2>,
 <Record review_id='92' occurrences=2>,
 <Record review_id='107' occurrences=2>,
 <Record review_id='12' occurrences=2>]


We got the id "None" that appears 19 times as we saw earlier, and then another 3111 ids that appear 2 times each, we need to see if they are total duplicates:

In [74]:
query = """
    MATCH (r:REVIEWS)
    WITH r.id AS review_id, COLLECT(r) AS duplicates
    WHERE SIZE(duplicates) = 2
    WITH duplicates[0] AS r1, duplicates[1] AS r2
    WHERE r1 <> r2 AND properties(r1) = properties(r2)
    RETURN COUNT(*) AS duplicate_count
"""
result = execute_read(driver, query)
pprint(result)

[<Record duplicate_count=3111>]


this query returns only the IDs of reviews that are exact duplicates (same id appearing 2 and same properties), so they are duplicates.

> BREWERIES analysis

In [75]:
query = """
    MATCH (b:BREWERIES)
    WITH b.id AS brewery_id, b.name AS brewery_name, b.state AS state, 
        b.notes AS notes, b.types AS types, COUNT(*) AS occurrences
    WHERE occurrences > 1
    RETURN brewery_id, brewery_name, state, notes, types, occurrences
    ORDER BY occurrences DESC
    limit 10;
"""
result = execute_read(driver, query)
pprint(result)

[<Record brewery_id='32541' brewery_name='Coachella Valley Brewing Co' state='CA' notes='No notes at this time.' types='Brewery, Bar, Beer-to-go' occurrences=2>,
 <Record brewery_id='44736' brewery_name="Beef 'O' Brady's" state='FL' notes='No notes at this time.' types='Bar, Eatery' occurrences=2>,
 <Record brewery_id='23372' brewery_name='Broadway Wine Merchant' state='OK' notes='No notes at this time.' types='Store' occurrences=2>,
 <Record brewery_id='35328' brewery_name='Brighton Beer Dispensary (DUPLICATE)' state='GB2' notes='Duplicate of https://www.beeradvocate.com/beer/profile/36929' types='Bar, Eatery' occurrences=2>,
 <Record brewery_id='31561' brewery_name="Teddy's Tavern" state='WA' notes='No notes at this time.' types='Bar, Beer-to-go' occurrences=2>,
 <Record brewery_id='35975' brewery_name='Modus Operandi Brewing Co.' state='nan' notes='No notes at this time.' types='Brewery, Bar, Eatery, Beer-to-go' occurrences=2>,
 <Record brewery_id='5618' brewery_name='Hops! Beer Res

we have 50 347 occurences that happen 2 times each. This gives us the total=100694. but also we notice the name with (DUPLICATE), so we will check that, and also we only have 45245 unique brewery_name so its possible that some names are under different ids (or ther properties like the state)

In [76]:
query = """
    MATCH (b:BREWERIES)
    WHERE b.name CONTAINS "DUPLICATE"
    RETURN COUNT(b) AS duplicate_count;
"""
result = execute_read(driver, query)
pprint(result)


[<Record duplicate_count=296>]


In [77]:
query = """
    MATCH (b1:BREWERIES)
    MATCH (b2:BREWERIES)
    WHERE b1.name = b2.name 
    AND (b1.id <> b2.id OR b1.state <> b2.state OR b1.notes <> b2.notes OR b1.types <> b2.types)
    WITH DISTINCT b1.name AS brewery_name, COUNT(*) AS count_variations
    RETURN brewery_name, count_variations
    ORDER BY count_variations DESC
    limit 10;
"""
result = execute_read(driver, query)
pprint(result)

[<Record brewery_name='Whole Foods Market' count_variations=104328>,
 <Record brewery_name='Total Wine & More' count_variations=85848>,
 <Record brewery_name='Cost Plus World Market' count_variations=55224>,
 <Record brewery_name='Mellow Mushroom' count_variations=51528>,
 <Record brewery_name="Trader Joe's" count_variations=30624>,
 <Record brewery_name='Old Chicago' count_variations=24024>,
 <Record brewery_name="BJ's Restaurant & Brewhouse" count_variations=18224>,
 <Record brewery_name='World of Beer' count_variations=17160>,
 <Record brewery_name='Yard House' count_variations=14640>,
 <Record brewery_name='Wegmans' count_variations=13224>]


> BEERS analysis

In [78]:
query = """
    MATCH (b:BEERS)
    WITH b.id AS beer_id, b.name AS beer_name, b.abv AS abv, 
        b.state AS state, b.availability AS availability, COUNT(*) AS occurrences
    WHERE occurrences > 1
    RETURN beer_id, beer_name, abv, state, availability, occurrences
    ORDER BY occurrences DESC
    limit 10;
"""
result = execute_read(driver, query)
pprint(result)

[<Record beer_id='8036' beer_name='World Burp Beer 2002' abv='5.5' state='nan' availability=' Limited (brewed once)' occurrences=2>,
 <Record beer_id='265827' beer_name='Addison' abv='5.5' state='PA' availability=' Rotating' occurrences=2>,
 <Record beer_id='110318' beer_name='Schwartz Hop' abv='6.0' state='CA' availability=' Limited (brewed once)' occurrences=2>,
 <Record beer_id='138971' beer_name='Porter' abv='5.3' state='NC' availability=' Rotating' occurrences=2>,
 <Record beer_id='199068' beer_name='Eastwood - Equinox' abv='8.2' state='CA' availability=' Limited (brewed once)' occurrences=2>,
 <Record beer_id='80010' beer_name='Joy Division Series: No Love Lost' abv='7.8' state='PA' availability=' Limited (brewed once)' occurrences=2>,
 <Record beer_id='55175' beer_name='Samuel Adams New World Lager' abv='nan' state='MA' availability=' Limited (brewed once)' occurrences=2>,
 <Record beer_id='55' beer_name='Redhook Double Black Stout' abv='7.0' state='NH' availability=' Rotating' 

We have 58 873 entries with 2 occurences each and 300 000 with 1 occurence (using WHERE occurrences = 1), this gives us the total 417746. But then again, we only have 298567 unique beers names, so same names can be under different ids, or different states...

In [79]:
query = """
    MATCH (b1:BEERS)
    MATCH (b2:BEERS)
    WHERE b1.name = b2.name 
    AND (b1.id <> b2.id OR b1.state <> b2.state OR b1.abv <> b2.abv OR b1.availability <> b2.availability)
    WITH DISTINCT b1.name AS beer_name, COUNT(*) AS count_variations
    RETURN beer_name, count_variations
    ORDER BY count_variations DESC
    LIMIT 10;
"""
result = execute_read(driver, query)
pprint(result)

[<Record beer_name='Oktoberfest' count_variations=776788>,
 <Record beer_name='IPA' count_variations=534894>,
 <Record beer_name='Pale Ale' count_variations=523244>,
 <Record beer_name='Hefeweizen' count_variations=300708>,
 <Record beer_name='Oatmeal Stout' count_variations=258440>,
 <Record beer_name='Porter' count_variations=186934>,
 <Record beer_name='Saison' count_variations=177540>,
 <Record beer_name='India Pale Ale' count_variations=126284>,
 <Record beer_name='Pilsner' count_variations=120662>,
 <Record beer_name='Brown Ale' count_variations=100062>]


> CITIES analysis

In [80]:
query = """ 
    MATCH (c:CITIES)
    RETURN c.name, COUNT(*) AS occurrences
    ORDER BY occurrences DESC
    LIMIT 10;
    """
result = execute_read(driver, query)
pprint(result)

[<Record c.name='Thousand Palms' occurrences=2>,
 <Record c.name='Plant City' occurrences=2>,
 <Record c.name='Oklahoma City' occurrences=2>,
 <Record c.name='Brighton' occurrences=2>,
 <Record c.name='Seattle' occurrences=2>,
 <Record c.name='Mona Vale' occurrences=2>,
 <Record c.name='Riccione (RN)' occurrences=2>,
 <Record c.name='Belfast' occurrences=2>,
 <Record c.name='Destin' occurrences=2>,
 <Record c.name='Erpe-Mere' occurrences=2>]


Every city is duplicated (occurrences = 2 for all)

> COUNTRIES analysis

In [81]:
query = """ 
    MATCH (c:COUNTRIES)
    RETURN c.name, COUNT(*) AS occurrences
    ORDER BY occurrences DESC
    limit 10;
    """
result = execute_read(driver, query)
pprint(result)

[<Record c.name='US' occurrences=2>,
 <Record c.name='GB' occurrences=2>,
 <Record c.name='AU' occurrences=2>,
 <Record c.name='IT' occurrences=2>,
 <Record c.name='CA' occurrences=2>,
 <Record c.name='GR' occurrences=2>,
 <Record c.name='FR' occurrences=2>,
 <Record c.name='AT' occurrences=2>,
 <Record c.name='ES' occurrences=2>,
 <Record c.name='BE' occurrences=2>]


Every country is duplicated (occurrences = 2 for all)

#### Data Quality Report

1. **Solve Naming Consistency Issues**: `STYLE` and `USER` ➡️ `STYLES` and `USERS` ✅

2. **Incorrect Relationship Directions**

- `(:REVIEWS)-[:POSTED]->(:USER)` **should be corrected to:**
  - `(:USER)-[:POSTED]->(:REVIEWS)` ✅
- `(:BEERS)-[:REVIEWED]->(:REVIEWS)` **should be corrected to:**
  - `(:REVIEWS)-[:REVIEWED]->(:BEERS)` ✅

3. **Unusual Relationships**
- **CITIES linked to CITIES via `IN` relationship is unusual.**- This **does not represent hierarchical regions** (e.g., no neighborhoods inside cities). 
- **Potential redundancy:** The direct `IN` relationship between breweries and countries **might be unnecessary**. If a brewery is assigned to a city, the country can be inferred.

4. **Orphaned Records and Duplicate Entities**

| Label     | Orphaned Count | Total Count | Unique Count (by name) | Total Duplicates (2 times) |Notes |Solved?          |
| --------- | -------------- | ----------- | ------------ | ---------- |---------- |---------- |
| USER      | 17,290         | 123,935     | 123,935      | 0          | No duplicates but we have Orphaned nodes         |✅           |
| COUNTRIES | 200            | 400         | 200          | 200        |Orphaned match duplicated          |✅           |
| CITIES    | 11,665         | 23,330      | 11,665       | 11,665     |Orphaned match duplicated           |✅           |
| BREWERIES | 50,347         | 100,694     | 45,245       | 50,347      |Orphaned match duplicated           |✅           |
| BEERS     | 58,873         | 417,746     | 298,567      | 58, 873    |Orphaned match duplicated           |✅           |

- **COUNTRIES and CITIES:** The number of orphaned records matches exactly with the number of duplicates found.
- **BREWERIES and BEERS:** Each orphaned record appears **twice**, confirming they are duplicated.

5. **Duplicate Reviews**

- `id = "None"` appears **19 times**. ✅
- `3,111` reviews appear **twice with the same properties**. ✅

**Breweries Containing 'DUPLICATE' in Name**: `296` ✅

`Notes`: Check the remaining duplicate names for Breweries and Beers after the first treatment. ✅

The CRUD references will be present across the cells.



### <div class="alert-warning" style="background-color: #FFF9C4; color: black; padding: 10px;"><a id="adjust-the-database"></a>2. Adjust the Database and Mention the Reasoning Behind the Adjustments </div>

Back to [Index](#index)

#### 2.1. Remove Orphaned Nodes (Nodes Without Any Relationships)

Orphaned nodes, which are nodes without relationships to any other nodes in the database, are unnecessary for the current data model and analysis. Removing these orphaned nodes helps to simplify the database, ensuring that only relevant data is kept. This reduces the overall size of the database, which can lead to better performance, faster query execution, and a clearer, more manageable schema for the given task. Also, we will solve the duplicates problem in some labels.

> DELETE Orphaned COUNTRIES

In [82]:
query = """ 
    MATCH (c:COUNTRIES)
    WHERE NOT EXISTS( (c)-[]-() ) 
    DETACH DELETE c;
    """
result = execute_write(driver, query)
pprint(result)

[]


In [83]:
query = """ 
    MATCH (c:COUNTRIES)
    WITH c.name AS country_name, COUNT(*) AS occurrences
    WHERE occurrences > 1
    RETURN country_name, occurrences;
    """
result = execute_read(driver, query)
pprint(result)

[]


Duplicates solved

> DELETE Orphaned CITIES

In [84]:
query = """ 
    MATCH (c:CITIES)
    WHERE NOT EXISTS( (c)-[]-() ) 
    DETACH DELETE c;
    """
result = execute_write(driver, query)
pprint(result)

[]


In [85]:
query = """ 
    MATCH (c:CITIES)
    WITH c.name AS city_name, COUNT(*) AS occurrences
    WHERE occurrences > 1
    RETURN city_name, occurrences;
    """
result = execute_read(driver, query)
pprint(result)

[]


Duplicates solved

> DELETE Orphaned BREWERIES

In [86]:
query = """ 
    MATCH (b:BREWERIES)
    WHERE NOT EXISTS( (b)-[]-() )
    DETACH DELETE b;
    """
result = execute_write(driver, query)
pprint(result)

[]


In [87]:
query = """
    MATCH (b:BREWERIES)
    WITH b.id AS brewery_id, b.name AS brewery_name, b.state AS state, 
        b.notes AS notes, b.types AS types, COUNT(*) AS occurrences
    WHERE occurrences > 1
    RETURN brewery_id, brewery_name, state, notes, types, occurrences
    ORDER BY occurrences DESC;
"""
result = execute_read(driver, query)
pprint(result)

[]


Total Duplicates solved

> DELETE Orphaned BEERS

In [88]:
query = """ 
    MATCH (b:BEERS)
    WHERE NOT EXISTS( (b)-[]-() )
    DETACH DELETE b;
    """
result = execute_write(driver, query)
pprint(result)

[]


In [89]:
query = """
    MATCH (b:BEERS)
    WITH b.id AS beer_id, b.name AS beer_name, b.abv AS abv, 
        b.state AS state, b.availability AS availability, COUNT(*) AS occurrences
    WHERE occurrences > 1
    RETURN beer_id, beer_name, abv, state, availability, occurrences
    ORDER BY occurrences DESC;
"""
result = execute_read(driver, query)
pprint(result)

[]


Total Duplicates solved

> DELETE Orphaned USER

In [90]:
query = """ 
    MATCH (u:USER)
    WHERE NOT EXISTS( (u)-[]-() )
    DETACH DELETE u;
    """
result = execute_write(driver, query)
pprint(result)

[]


> DELETE Orphaned REVIEWS

In [91]:
query = """ 
    MATCH (r:REVIEWS)
    WHERE NOT EXISTS( (r)-[]-() )
    DETACH DELETE r;
    """
result = execute_write(driver, query)
pprint(result)

[]


In [92]:
query = """
    MATCH (r:REVIEWS)
    WITH r.id AS review_id, COUNT(*) AS occurrences
    WHERE occurrences > 1
    RETURN review_id, occurrences
    ORDER BY occurrences DESC
    limit 10;
"""
result = execute_read(driver, query)
pprint(result)

[<Record review_id=None occurrences=19>]


Also solved the total duplicates in reviews. But we still have the none values.

In [93]:
query = """
    MATCH (n)
    WHERE NOT EXISTS( (n)-[]-() )
    RETURN labels(n) AS Label, COUNT(n) AS OrphanedCount
"""
result = execute_read(driver, query)
pprint(result)

[]


⬆️ all fine now.

#### 2.2. Duplicates (None values for REVIEWS and partial duplicates for REVIEWS; BEERS and BREWERIES)

Back to [Index](#index)

> Delete REVIEWS with id = "None"

In [94]:
#first, since we notice that not all names and ids are unique, its best to assert the schema - UPDATE

query = """ 
    CALL apoc.schema.assert({}, {});
    """
result = execute_write(driver, query)
pprint(result)

[<Record label='COUNTRIES' key='name' keys=['name'] unique=False action='DROPPED'>,
 <Record label='CITIES' key='name' keys=['name'] unique=False action='DROPPED'>,
 <Record label='BREWERIES' key='id' keys=['id'] unique=False action='DROPPED'>,
 <Record label='BEERS' key='id' keys=['id'] unique=False action='DROPPED'>,
 <Record label='REVIEWS' key='id' keys=['id'] unique=False action='DROPPED'>,
 <Record label='STYLE' key='name' keys=['name'] unique=False action='DROPPED'>,
 <Record label='USER' key='name' keys=['name'] unique=False action='DROPPED'>]


In [95]:
#DELETE

query = """ 
    MATCH (r:REVIEWS)
    WHERE r.id IS NULL OR r.id = "None"
    DETACH DELETE r;
    """
result = execute_write(driver, query)
pprint(result)

[]


Due to memory issues, we were unable to fully complete the detection of partial duplicates in the REVIEWS. Although we attempted to increase the allocated memory, the computer kept crashing, preventing us from identifying all partial duplicates among the reviews. However, we were able to resolve the total (exact) duplicates.

Additionally, while trying to process duplicates in batches to avoid memory overload, we encountered another issue: the batch processing method did not retrieve all duplicates correctly. Since each batch only processed a subset of the data, it failed to detect certain duplicates that might have been present in different batches. As a result, the approach was not effective for our use case.

> BREWERIES also has duplicates if we dont consider the id and has some nodes with "Duplicate" in the name: what we want to see is that if we remove the " (DUPLICATE)" from the name, and considering the normal name, types and state as properties (the id and the notes will be different because we have some notes saying "this is a duplicate of this"...)

In [96]:
# CREATE - the composite index helps in finding duplicates quickly (we use the properties that should compose a unique record)
query = """ 
    CREATE INDEX breweries_composite_index IF NOT EXISTS FOR (b:BREWERIES) ON (b.name, b.types, b.state, b.notes);
"""
result = execute_write(driver, query)
pprint(result)

[]


In [97]:
#READ
query = """
    SHOW INDEXES YIELD name, properties WHERE name = "breweries_composite_index"
"""
result = execute_read(driver, query)
pprint(result)

[<Record name='breweries_composite_index' properties=['name', 'types', 'state', 'notes']>]


In [98]:
#READ
query = """ 
    MATCH (b:BREWERIES)
    WITH b.name AS name, b.types AS types, b.state AS state, b.notes AS notes, COUNT(*) AS occurrences
    WHERE occurrences > 1
    RETURN name, types, state, notes, occurrences
    ORDER BY occurrences DESC
    limit 10;
"""
result = execute_read(driver, query)
pprint(result)

[<Record name='Beverages & more!' types='Store' state='CA' notes='No notes at this time.' occurrences=51>,
 <Record name="Trader Joe's" types='Store' state='CA' notes='No notes at this time.' occurrences=43>,
 <Record name='Whole Foods Market' types='Store' state='CA' notes='No notes at this time.' occurrences=40>,
 <Record name='ABC Fine Wine & Spirits' types='Store' state='FL' notes='No notes at this time.' occurrences=39>,
 <Record name="Binny's Beverage Depot" types='Store' state='IL' notes='No notes at this time.' occurrences=30>,
 <Record name='Giant Eagle' types='Store' state='OH' notes='No notes at this time.' occurrences=29>,
 <Record name='Cost Plus World Market' types='Store' state='CA' notes='No notes at this time.' occurrences=29>,
 <Record name='Total Wine & More' types='Store' state='FL' notes='No notes at this time.' occurrences=25>,
 <Record name='Beverages & More!' types='Store' state='CA' notes='No notes at this time.' occurrences=24>,
 <Record name='Wegmans' types='

these breweries are considered duplicates because they have the same name, types, and state, even though their IDs are different. Now, we need to merge them:

In [99]:
# DELETE: we merge the duplicates
query = """
    CALL {
        MATCH (b:BREWERIES)
        WHERE b.name IS NOT NULL
        WITH b.name AS name, b.types AS types, b.state AS state, COLLECT(b) AS breweries
        WHERE SIZE(breweries) > 1
        RETURN breweries
    }

    UNWIND breweries AS brewery

    WITH brewery, breweries[0] AS main  // Main node to maintain
    WHERE brewery <> main

    CALL apoc.refactor.mergeNodes([main, brewery], {properties: {name: 'keep', types: 'keep', state: 'keep', notes: 'keep'}, mergeRels: true})
    YIELD node

    RETURN COUNT(node) AS Merged_Breweries;
"""
result = execute_write(driver, query)
pprint(result)

[<Record Merged_Breweries=2877>]


> Now, the records with DUPLICATE, we first see if removing the "DUPLICATE" from the name would we have duplicates

In [100]:
#READ
query = """
    MATCH (b:BREWERIES)
    WITH TRIM(REPLACE(b.name, " (DUPLICATE)", "")) AS NormalName, b.types AS Type, b.state AS State, b
    WITH NormalName, Type, State, COUNT(b) AS TotalDuplicates, COLLECT(b.name) AS DuplicateNames
    WHERE TotalDuplicates > 1
    RETURN NormalName, Type, State, DuplicateNames, TotalDuplicates
    ORDER BY TotalDuplicates DESC
    LIMIT 10;
"""
result = execute_read(driver, query)
pprint(result)

[<Record NormalName='Emerald Hill Brewery' Type='Brewery' State='nan' DuplicateNames=['Emerald Hill Brewery (DUPLICATE)', 'Emerald Hill Brewery'] TotalDuplicates=2>,
 <Record NormalName="Joe's Tap Room" Type='Bar, Eatery' State='FL' DuplicateNames=["Joe's Tap Room (DUPLICATE)", "Joe's Tap Room"] TotalDuplicates=2>,
 <Record NormalName='Pie Whole' Type='Bar, Eatery' State='nan' DuplicateNames=['Pie Whole', 'Pie Whole (DUPLICATE)'] TotalDuplicates=2>,
 <Record NormalName='The Pub' Type='Bar, Eatery' State='FL' DuplicateNames=['The Pub', 'The Pub (DUPLICATE)'] TotalDuplicates=2>,
 <Record NormalName='Upstate Brewing Company' Type='Brewery, Beer-to-go' State='NY' DuplicateNames=['Upstate Brewing Company', 'Upstate Brewing Company (DUPLICATE)'] TotalDuplicates=2>,
 <Record NormalName='Bull City Homebrew' Type='Homebrew' State='NC' DuplicateNames=['Bull City Homebrew (DUPLICATE)', 'Bull City Homebrew'] TotalDuplicates=2>,
 <Record NormalName='Fairhope Brewing Company' Type='Brewery, Bar' Sta

In [101]:
query = """ 
    MATCH (b:BREWERIES)
    WHERE b.name = "Emerald Hill Brewery"  // 
      AND b.types = "Brewery" 
      AND b.state = "nan"
    RETURN b;
"""
result = execute_read(driver, query)
pprint(result)

[<Record b=<Node element_id='22932' labels=frozenset({'BREWERIES'}) properties={'types': 'Brewery', 'notes': 'No notes at this time.', 'name': 'Emerald Hill Brewery', 'id': '14687', 'state': 'nan'}>>]


In [102]:
query = """ 
    MATCH (b:BREWERIES)
    WHERE b.name = "Emerald Hill Brewery (DUPLICATE)"  
      AND b.types = "Brewery" 
      AND b.state = "nan"
    RETURN b;
"""
result = execute_read(driver, query)
pprint(result)

[<Record b=<Node element_id='12658' labels=frozenset({'BREWERIES'}) properties={'types': 'Brewery', 'notes': 'No notes at this time.', 'name': 'Emerald Hill Brewery (DUPLICATE)', 'id': '14455', 'state': 'nan'}>>]


In [103]:
#DELETE: merge the cases
query = """
    CALL {
        MATCH (b:BREWERIES)
        WITH TRIM(REPLACE(b.name, " (DUPLICATE)", "")) AS NormalName, b.types AS Type, b.state AS State, b
        WITH NormalName, Type, State, COLLECT(b) AS Breweries
        WHERE SIZE(Breweries) > 1
        RETURN Breweries
    }

    WITH [b IN Breweries WHERE NOT b.name CONTAINS "(DUPLICATE)"] AS CleanNames, Breweries
    WITH CASE 
        WHEN SIZE(CleanNames) > 0 THEN CleanNames[0] // keeps the name without duplicate in it 
        ELSE Breweries[0] 
    END AS MainBrewery, Breweries

    UNWIND Breweries AS Brewery
    WITH Brewery, MainBrewery
    WHERE Brewery <> MainBrewery

    CALL apoc.refactor.mergeNodes([MainBrewery, Brewery], {properties: {name: 'keep', types: 'keep', state: 'keep', notes: 'combine'}, mergeRels: true})
    YIELD node
    RETURN COUNT(node) AS Merged_Breweries;
"""
result = execute_write(driver, query)
pprint(result)

[<Record Merged_Breweries=56>]


In [104]:
query = """ 
    MATCH (b:BREWERIES)
    WITH b.name AS name, b.types AS types, b.state AS state, b.notes AS notes, COUNT(*) AS occurrences
    WHERE occurrences > 1
    RETURN name, types, state, notes, occurrences
    ORDER BY occurrences DESC;
"""
result = execute_read(driver, query)
pprint(result)

[]


In [105]:
#test for (DUPLICATE)
query = """ 
    MATCH (b:BREWERIES)
    WHERE b.name = "Emerald Hill Brewery (DUPLICATE)"  // Emerald Hill Brewery (DUPLICATE)
      AND b.types = "Brewery" 
      AND b.state = "nan"
    RETURN b;
"""
result = execute_read(driver, query)
pprint(result)

[]


> BEERS DUPLICATES, same approach

In [106]:
# CREATE 
query = """ 
    CREATE INDEX beers_composite_index IF NOT EXISTS FOR (b:BEERS) ON (b.abv, b.name, b.state, b.availability, b.retired, b.brewery_id, b.notes);
"""
result = execute_write(driver, query)
pprint(result)

[]


In [107]:
#READ
query = """
    SHOW INDEXES YIELD name, properties WHERE name = "beers_composite_index";
"""
result = execute_read(driver, query)
pprint(result)

[<Record name='beers_composite_index' properties=['abv', 'name', 'state', 'availability', 'retired', 'brewery_id', 'notes']>]


In [108]:
#READ
query = """ 
    MATCH (b:BEERS)
    WITH b.abv AS abv, b.name AS name, b.state AS state, 
        b.availability AS availability, b.retired AS retired, 
        b.brewery_id AS brewery_id, b.notes AS notes, 
        COUNT(*) AS occurrences
    WHERE occurrences > 1
    RETURN name, abv, state, availability, retired, brewery_id, notes, occurrences
    ORDER BY occurrences DESC
    LIMIT 10;
"""
result = execute_read(driver, query)
pprint(result)

[<Record name='Original' abv='5.0' state='nan' availability=' Rotating' retired='f' brewery_id='9055' notes='No notes at this time.' occurrences=3>,
 <Record name='02' abv='5.5' state='GB2' availability=' Rotating' retired='f' brewery_id='33192' notes='No notes at this time.' occurrences=3>,
 <Record name='Sequoia Stout' abv='17.0' state='CA' availability=' Limited (brewed once)' retired='f' brewery_id='42975' notes='No notes at this time.' occurrences=3>,
 <Record name='05' abv='6.5' state='GB2' availability='Rotating' retired='f' brewery_id='33192' notes='No notes at this time.' occurrences=3>,
 <Record name='05' abv='7.2' state='GB2' availability=' Rotating' retired='f' brewery_id='33192' notes='No notes at this time.' occurrences=3>,
 <Record name='01' abv='5.5' state='GB2' availability=' Limited (brewed once)' retired='f' brewery_id='33192' notes='No notes at this time.' occurrences=2>,
 <Record name='Stella Baltic Porter' abv='9.9' state='CO' availability=' Limited (brewed once)'

In [109]:
#DELETE
query = """
    CALL {
        MATCH (b:BEERS)
        WHERE b.name IS NOT NULL
        WITH b.abv AS abv, b.name AS name, b.state AS state, 
            b.availability AS availability, b.retired AS retired, 
            b.brewery_id AS brewery_id, COLLECT(b) AS beers
        WHERE SIZE(beers) > 1
        RETURN beers}

    UNWIND beers AS beer
    WITH beer, beers[0] AS main  // Main node to maintain
    WHERE beer <> main

    CALL apoc.refactor.mergeNodes(
        [main, beer], 
        {properties: {abv: 'keep', name: 'keep', state: 'keep', 
                    availability: 'keep', retired: 'keep', 
                    brewery_id: 'keep', notes: 'keep'}, // we can use keep bc they are the same 
        mergeRels: true})
    YIELD node

    RETURN COUNT(node) AS Merged_Beers;
"""
result = execute_write(driver, query)
pprint(result)

[<Record Merged_Beers=121>]


#### 2.3. UPDATE: Relationship Directions

For this part we were unable to change the relationships directions initially due to memory issues and computational power sooo, we found a solution using the apoc. that is another file we added in the pugins as we explain in the intro.

Back to [Index](#index)

✅ Change POSTED to (:USERS)-[:POSTED]->(:REVIEWS) - a user posts the review, not the other way around

In [110]:
query = """ 
    CALL apoc.periodic.iterate(
    "MATCH (u:USER)<-[p:POSTED]-(r:REVIEWS) RETURN u, p, r",
    "CREATE (u)-[new:POSTED]->(r) SET new += p DELETE p",
    {batchSize: 10000, parallel: false});
"""
result = execute_write(driver, query)
pprint(result)

[<Record batches=254 total=2538044 timeTaken=493 committedOperations=2538044 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 254, 'committed': 254, 'failed': 0, 'errors': {}} operations={'total': 2538044, 'committed': 2538044, 'failed': 0, 'errors': {}} wasTerminated=False failedParams={} updateStatistics={'nodesDeleted': 0, 'labelsAdded': 0, 'relationshipsCreated': 2538044, 'nodesCreated': 0, 'propertiesSet': 0, 'relationshipsDeleted': 2538044, 'labelsRemoved': 0}>]


✅ Change REVIEWED to (:REVIEWS)-[:REVIEWED]->(:BEERS) - a review reviews a beer, not the other way around (we need to make this with more than one batch number in other to cover all relations - 1st 10000, them 1000, then 100, then 10, then 1)

In [111]:
query = """ 
    CALL apoc.periodic.iterate(
    "MATCH (b:BEERS)-[rel:REVIEWED]->(r:REVIEWS) RETURN b, rel, r",
    "CREATE (r)-[new:REVIEWED]->(b) SET new += rel DELETE rel",
    {batchSize: 1, parallel: false});
"""
result = execute_write(driver, query)
pprint(result)

[<Record batches=2537991 total=2537991 timeTaken=2404 committedOperations=2537991 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 2537991, 'committed': 2537991, 'failed': 0, 'errors': {}} operations={'total': 2537991, 'committed': 2537991, 'failed': 0, 'errors': {}} wasTerminated=False failedParams={} updateStatistics={'nodesDeleted': 0, 'labelsAdded': 0, 'relationshipsCreated': 2537991, 'nodesCreated': 0, 'propertiesSet': 0, 'relationshipsDeleted': 2537991, 'labelsRemoved': 0}>]


In [112]:
query = """
    MATCH (a)-[r]->(b)
    RETURN DISTINCT labels(a) AS StartNode, type(r) AS Relationship, labels(b) AS EndNode, COUNT(*) AS count
    ORDER BY count DESC;
"""
result = execute_read(driver, query)
pprint(result)

[<Record StartNode=['USER'] Relationship='POSTED' EndNode=['REVIEWS'] count=2538044>,
 <Record StartNode=['REVIEWS'] Relationship='REVIEWED' EndNode=['BEERS'] count=2537991>,
 <Record StartNode=['BEERS'] Relationship='HAS_STYLE' EndNode=['STYLE'] count=358768>,
 <Record StartNode=['BREWERIES'] Relationship='BREWED' EndNode=['BEERS'] count=358752>,
 <Record StartNode=['BREWERIES'] Relationship='IN' EndNode=['CITIES'] count=49992>,
 <Record StartNode=['CITIES'] Relationship='IN' EndNode=['COUNTRIES'] count=12077>]


#### 2.4. Remove Invalid Relationships (CITIES linked to CITIES)

This part does not appear imediatly corrected in the graph, but when we restart the docker eventually appeared with the corrections.

In [113]:
query = """ 
    MATCH (a)-[r:IN]->(b)
    WHERE NOT 
        ((a:BREWERIES AND b:CITIES) OR
        (a:CITIES AND b:COUNTRIES))
    DELETE r;
    """
result = execute_write(driver, query)
pprint(result)

[]


In [114]:
query = """ 
    MATCH (a)-[r:IN]->(b)
    RETURN DISTINCT labels(a), labels(b), COUNT(r);
    """
result = execute_read(driver, query)
pprint(result)

[<Record labels(a)=['BREWERIES'] labels(b)=['CITIES'] COUNT(r)=49992>,
 <Record labels(a)=['CITIES'] labels(b)=['COUNTRIES'] COUNT(r)=12077>]


#### 2.5. Remove Redundant BREWERIES → COUNTRIES Relationships

Since each BREWERIES is linked to a CITIES, we can infer COUNTRIES, making the direct link unnecessary (and we saw that they dont have connections).

In [115]:
query = """ 
    MATCH (b:BREWERIES)-[i:IN]->(c:COUNTRIES)
    DELETE i;
    """
result = execute_write(driver, query)
pprint(result)

[]


#### 2.6. Solve Label Naming Consistency Issues (this is mostly a display issue rather than a functional problem, but is a good pratice)

Again, the old labels only disapear when we restarted the docker.

Back to [Index](#index)

In [116]:
query = """CALL apoc.refactor.rename.label("STYLE", "STYLES");"""
result = execute_write(driver, query)
pprint(result)

[<Record batches=1 total=113 timeTaken=0 committedOperations=113 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 1, 'committed': 1, 'failed': 0, 'errors': {}} operations={'total': 113, 'committed': 113, 'failed': 0, 'errors': {}} constraints=[] indexes=[]>]


In [117]:
query = """CALL apoc.refactor.rename.label("USER", "USERS");"""
result = execute_write(driver, query)
pprint(result)

[<Record batches=2 total=106645 timeTaken=0 committedOperations=106645 failedOperations=0 failedBatches=0 retries=0 errorMessages={} batch={'total': 2, 'committed': 2, 'failed': 0, 'errors': {}} operations={'total': 106645, 'committed': 106645, 'failed': 0, 'errors': {}} constraints=[] indexes=[]>]


In [118]:
query = """
    MATCH (n:STYLE)
    REMOVE n:STYLE;
"""
result = execute_write(driver, query)
pprint(result)

[]


In [119]:
query = """
    MATCH (n:USER)
    REMOVE n:USER;
"""
result = execute_write(driver, query)
pprint(result)

[]


In [120]:
query = """
    MATCH (n)
    RETURN labels(n) AS Label, COUNT(*) AS Count
    ORDER BY Count DESC;
"""
result = execute_read(driver, query)
pprint(result)

[<Record Label=['REVIEWS'] Count=2538044>,
 <Record Label=['BEERS'] Count=358752>,
 <Record Label=['USERS'] Count=106645>,
 <Record Label=['BREWERIES'] Count=47414>,
 <Record Label=['CITIES'] Count=11665>,
 <Record Label=['COUNTRIES'] Count=200>,
 <Record Label=['STYLES'] Count=113>]


![Corrected Database Schema Visualization](https://raw.githubusercontent.com/CatarinaGN/Big-Data-Modelling-and-Management/e09c6c029b94fb7b5a4024d01315253f48cc3609/corrected%20schema.png)

This is from the http://localhost:7474/browser/ suing CALL apoc.meta.graph();


### <div class="alert-warning" style="background-color: #FFF9C4; color: black; padding: 10px;"> <a id="analytics-department"></a>3. Analytics Department Biweekly Reporting Requirements </div>

Back to [Index](#index)

#### <a id="beer-with-most-reviews"></a>3.1. How many reviews does the beer with the most reviews have?

In [121]:
query_q1 = """
        MATCH (re:REVIEWS)-[r:REVIEWED]->(be:BEERS)
        RETURN be.name AS Beer, count(r) AS review_count
        ORDER BY review_count DESC
        LIMIT 1
    """

result = execute_read(driver, query_q1)
pprint(result)

[<Record Beer='IPA' review_count=8771>]


#### <a id="three-users-most-reviews"></a>3.2. Which three users wrote the most reviews about beers?

In [122]:
query_q2 = """
    MATCH (u:USERS)-[:POSTED]->(r:REVIEWS)
    RETURN u.name AS User, COUNT(r) AS review_count
    ORDER BY review_count DESC
    LIMIT 3;
"""
result = execute_read(driver, query_q2)
pprint(result)

[<Record User='Sammy' review_count=3756>,
 <Record User='acurtis' review_count=3403>,
 <Record User='kylehay2004' review_count=3368>]


#### <a id="beers-with-descriptions"></a>3.3. Find all beers that are described with the following words: 'fruit', 'complex', 'nutty', 'dark'.

In this question we consider all the beers that have all four words appear in the same review - all words are expected to be used together! "the review must contain 'fruit' AND 'complex' AND 'nutty' AND 'dark'."

In [123]:
query_q3 = """
    MATCH (b:BEERS)<-[:REVIEWED]-(r:REVIEWS)
    WHERE ALL(word IN ['fruit', 'complex', 'nutty', 'dark'] WHERE toLower(r.text) CONTAINS word)
    RETURN DISTINCT b.name AS Beer, COUNT(r) AS matching_reviews
    ORDER BY matching_reviews DESC;
"""
result = execute_read(driver, query_q3)
pprint(result)

[<Record Beer='Trappist Westvleteren 12 (XII)' matching_reviews=8>,
 <Record Beer='Palo Santo Marron' matching_reviews=8>,
 <Record Beer='Nut Brown Ale' matching_reviews=6>,
 <Record Beer='Trappistes Rochefort 10' matching_reviews=6>,
 <Record Beer='Trappistes Rochefort 8' matching_reviews=6>,
 <Record Beer='Abt 12' matching_reviews=6>,
 <Record Beer='Trappistes Rochefort 6' matching_reviews=5>,
 <Record Beer='Abbey Ale' matching_reviews=4>,
 <Record Beer='Celebrator' matching_reviews=4>,
 <Record Beer='Old Rasputin' matching_reviews=4>,
 <Record Beer="Fuller's 1845" matching_reviews=4>,
 <Record Beer='Wee Heavy' matching_reviews=4>,
 <Record Beer='Indian Brown Ale' matching_reviews=4>,
 <Record Beer='Porter' matching_reviews=4>,
 <Record Beer='Third Coast Old Ale' matching_reviews=4>,
 <Record Beer='Arrogant Bastard Ale' matching_reviews=3>,
 <Record Beer='Spaten Optimator' matching_reviews=3>,
 <Record Beer='Schneider Weisse Tap 6 Unser Aventinus' matching_reviews=3>,
 <Record Beer='

#### <a id="top-three-breweries"></a>3.4. Which top three breweries produce the largest variety of beer styles?

In [124]:
query_q4 = """
        MATCH (b:BREWERIES)-[:BREWED]->(be:BEERS)-[:HAS_STYLE]->(s:STYLES)
        RETURN b.name AS Brewery, count(DISTINCT s) AS unique_styles
        ORDER BY unique_styles DESC
        LIMIT 3
    """
result = execute_read(driver, query_q4)
pprint(result)

[<Record Brewery='Iron Hill Brewery & Restaurant' unique_styles=94>,
 <Record Brewery='Rock Bottom Restaurant & Brewery' unique_styles=93>,
 <Record Brewery='Goose Island Beer Co.' unique_styles=88>]


#### <a id="country-most-styles"></a>3.5. Which country produces the most beer styles?

In [125]:
query_q5 = """
    MATCH (c:COUNTRIES)<-[:IN]-(ct:CITIES)<-[:IN]-(br:BREWERIES)-[:BREWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLES)
    WITH c.name AS Country, COUNT(DISTINCT s.name) AS style_count
    RETURN Country, style_count
    ORDER BY style_count DESC
    LIMIT 1;
"""
result = execute_read(driver, query_q5)
pprint(result)

[<Record Country='US' style_count=113>]


### <div class="alert-warning" style="background-color: #FFF9C4; color: black; padding: 10px;"> <a id="optimizing-performance"></a>4. Optimizing Database Performance for Market Analysis Department</div>

Back to [Index](#index)

#### <a id="strongest-beers"></a>4.1. Using ABV score, find five strongest beers, display their ABV score and the corresponding brewery

In [126]:
query_q1_optimized = """
MATCH (b:BEERS)
WHERE toFloat(b.abv) IS NOT NULL AND NOT b.abv = "nan"
WITH b
MATCH (br:BREWERIES {id: b.brewery_id})
RETURN b.name AS beer_name, toFloat(b.abv) AS abv, br.name AS brewery_name
ORDER BY abv DESC
LIMIT 5
"""

result_q1_opt = execute_read(driver, query_q1_optimized)
pprint(result_q1_opt)

[<Record beer_name="Earache: World's Shortest Album" abv=100.0 brewery_name='1648 Brewing Company Ltd'>,
 <Record beer_name='Radiohead - OK Computer' abv=100.0 brewery_name='1648 Brewing Company Ltd'>,
 <Record beer_name='water' abv=100.0 brewery_name='Avondale Brewing Co.'>,
 <Record beer_name='Dark Reckoning' abv=80.0 brewery_name='Morgan Territory Brewing'>,
 <Record beer_name='Snake Venom' abv=67.5 brewery_name='Brewmeister'>]


In this step, we aim to identify the top 5 strongest beers by ABV (Alcohol By Volume) and display their respective breweries.  
Due to the absence of a direct relationship between beers and breweries in the graph, we leveraged the `brewery_id` attribute present in the BEERS node and matched it with the `id` of the BREWERIES node.

We also handled data consistency issues, filtering out invalid ABV values such as "nan" and converting the property to a numeric format for correct sorting.

The following Python code runs a Cypher query that retrieves the beer name, ABV score, and the brewery name, sorted by descending ABV.


#### <a id="top-5-beer-styles"></a>4.2. Find the top 5 distinct beer styles with the highest average score of smell and feel that were reviewed by the third most productive user.

In [127]:
#first we found the 3rd most Productive User (in terms of number of reviews)
query_3rd_user = """
MATCH (u:USERS)-[:POSTED]->(r:REVIEWS)
WITH u, COUNT(r) AS review_count
ORDER BY review_count DESC
SKIP 2 // Skip the top 2 users
LIMIT 1
RETURN u.name AS third_most_productive_user
"""
result_q2 = execute_read(driver, query_3rd_user)
pprint(result_q2)

[<Record third_most_productive_user='kylehay2004'>]


In [128]:
query_q2 = """
MATCH (u:USERS {name: "kylehay2004"})-[:POSTED]->(r:REVIEWS)-[:REVIEWED]->(b:BEERS)-[:HAS_STYLE]->(s:STYLES)
WHERE toFloat(r.smell) IS NOT NULL AND toFloat(r.feel) IS NOT NULL
RETURN s.name AS beer_style,
       round(avg(toFloat(r.smell)),2) AS avg_smell,
       round(avg(toFloat(r.feel)),2) AS avg_feel,
       round(avg(toFloat(r.smell) + toFloat(r.feel))/2, 2) AS combined_avg
ORDER BY combined_avg DESC
LIMIT 5
"""

result_q2 = execute_read(driver, query_q2)
pprint(result_q2)

[<Record beer_style='New England IPA' avg_smell=4.3 avg_feel=4.36 combined_avg=4.33>,
 <Record beer_style='American Imperial Stout' avg_smell=4.29 avg_feel=4.25 combined_avg=4.27>,
 <Record beer_style='Bière de Champagne / Bière Brut' avg_smell=4.25 avg_feel=4.25 combined_avg=4.25>,
 <Record beer_style='Belgian Fruit Lambic' avg_smell=4.25 avg_feel=4.19 combined_avg=4.22>,
 <Record beer_style='Belgian Gueuze' avg_smell=4.21 avg_feel=4.21 combined_avg=4.21>]


We retrieved the top 5 beer styles with the highest average of smell and feel scores, considering only the reviews made by the 3rd most active user (`kylehay2004`).  
To achieve this, we calculated the average smell and feel scores for each beer style and computed a combined average to rank the styles.

### <div class="alert-warning" style="background-color: #FFF9C4; color: black; padding: 10px;"> <a id="graph-algorithms"></a>5. Questions Using Graph Algorithms (gds)</div>

We will do the 5.1 and 5.2, the 5.3 was one of our initial choice but due to memory issues we were no able to run all of it, so we leave the code and the locial with 100 000 testing relations but we techinically do not consider it part of the 2 out of 4.

Back to [Index](#index)

#### <a id="similar-countries"></a>5.1. Which two countries are most similar when it comes to their top five most produced Beer styles? DONE

In [129]:
# DELETE - graph if exists
query_drop = """
CALL gds.graph.drop('Country_Beer_Style_Graph', false) YIELD graphName;
"""

try:
    result_drop = execute_read(driver, query_drop)
    pprint(result_drop)
except Exception as e:
    print(e)

[]


In [130]:
# CREATE - graph projection with only the top 5 beer styles per country
query_create = """
CALL gds.graph.project.cypher(
    'Country_Beer_Style_Graph',

    // Nodes: Countries and Styles
    'MATCH (c:COUNTRIES) RETURN id(c) AS id
     UNION
     MATCH (s:STYLES) RETURN id(s) AS id',

    // Relationships: Only top 5 beer styles per country, with ranked weight
    'MATCH (c:COUNTRIES)<-[:IN]-(city:CITIES)<-[:IN]-(brewery:BREWERIES)-[:BREWED]->(beer:BEERS)-[:HAS_STYLE]->(s:STYLES)
     WITH c, s, count(beer) AS beer_count
     ORDER BY beer_count DESC
     WITH c, collect({style: s, count: beer_count})[0..5] AS top_styles
     
     UNWIND top_styles AS style_data
     RETURN id(c) AS source, id(style_data.style) AS target, style_data.count AS weight'
);

"""

result_create = execute_write(driver, query_create)
pprint(result_create)

[<Record nodeQuery='MATCH (c:COUNTRIES) RETURN id(c) AS id\n     UNION\n     MATCH (s:STYLES) RETURN id(s) AS id' relationshipQuery='MATCH (c:COUNTRIES)<-[:IN]-(city:CITIES)<-[:IN]-(brewery:BREWERIES)-[:BREWED]->(beer:BEERS)-[:HAS_STYLE]->(s:STYLES)\n     WITH c, s, count(beer) AS beer_count\n     ORDER BY beer_count DESC\n     WITH c, collect({style: s, count: beer_count})[0..5] AS top_styles\n     \n     UNWIND top_styles AS style_data\n     RETURN id(c) AS source, id(style_data.style) AS target, style_data.count AS weight' graphName='Country_Beer_Style_Graph' nodeCount=313 relationshipCount=813 projectMillis=3786>]


In [131]:
# RETURN distinct Labels (n) - to check (Verifying the Graph Structure)
query = """
MATCH (n) 
WHERE head(Labels(n))='COUNTRIES' OR head(Labels(n))='STYLES' 
RETURN id(n) AS id
LIMIT 20;

"""

result_cosine = execute_read(driver, query)
pprint(result_cosine)

[<Record id=0>,
 <Record id=1>,
 <Record id=2>,
 <Record id=3>,
 <Record id=4>,
 <Record id=5>,
 <Record id=6>,
 <Record id=7>,
 <Record id=8>,
 <Record id=9>,
 <Record id=10>,
 <Record id=11>,
 <Record id=12>,
 <Record id=13>,
 <Record id=14>,
 <Record id=15>,
 <Record id=16>,
 <Record id=17>,
 <Record id=18>,
 <Record id=19>]


In [132]:
# Verifying Relationships
query = """
MATCH (c:COUNTRIES)<-[:IN]-(city:CITIES)<-[:IN]-(brewery:BREWERIES)-[:BREWED]->(beer:BEERS)-[:HAS_STYLE]->(s:STYLES)
WITH c, s, count(beer) AS beer_count
ORDER BY beer_count DESC
WITH c, collect({style: s, count: beer_count})[0..5] AS top_styles
UNWIND top_styles AS style_data
RETURN c AS country, style_data.style AS beer_style, style_data.count AS frequency
LIMIT 10;

"""

result_cosine = execute_read(driver, query)
pprint(result_cosine)

[<Record country=<Node element_id='1' labels=frozenset({'COUNTRIES'}) properties={'name': 'US'}> beer_style=<Node element_id='9494220' labels=frozenset({'STYLES'}) properties={'name': 'American IPA'}> frequency=39079>,
 <Record country=<Node element_id='1' labels=frozenset({'COUNTRIES'}) properties={'name': 'US'}> beer_style=<Node element_id='9494228' labels=frozenset({'STYLES'}) properties={'name': 'American Pale Ale (APA)'}> frequency=18223>,
 <Record country=<Node element_id='1' labels=frozenset({'COUNTRIES'}) properties={'name': 'US'}> beer_style=<Node element_id='9494236' labels=frozenset({'STYLES'}) properties={'name': 'American Imperial IPA'}> frequency=16630>,
 <Record country=<Node element_id='1' labels=frozenset({'COUNTRIES'}) properties={'name': 'US'}> beer_style=<Node element_id='9494241' labels=frozenset({'STYLES'}) properties={'name': 'Belgian Saison'}> frequency=15282>,
 <Record country=<Node element_id='1' labels=frozenset({'COUNTRIES'}) properties={'name': 'US'}> beer_

In [133]:
# Node Similarity Algorithm
query_cosine = """
CALL gds.nodeSimilarity.stream('Country_Beer_Style_Graph')
YIELD node1, node2, similarity

WITH gds.util.asNode(node1).name AS Country1, 
     gds.util.asNode(node2).name AS Country2, 
     similarity

RETURN Country1, Country2, similarity
ORDER BY similarity DESC
limit 1;
"""

result_cosine = execute_read(driver, query_cosine)
pprint(result_cosine)

[<Record Country1='US' Country2='PH' similarity=1.0>]


So the 2 most similar countries are the United States and the Philippines. But we notice something we can analyze further:

In [134]:
#Read the similarity reuslts distribution
query = """
CALL gds.nodeSimilarity.stream('Country_Beer_Style_Graph')
YIELD similarity
RETURN similarity, count(*) AS frequency
ORDER BY similarity;
"""

result_cosine = execute_read(driver, query)
pprint(result_cosine)

[<Record similarity=0.1111111111111111 frequency=25>,
 <Record similarity=0.125 frequency=6>,
 <Record similarity=0.14285714285714285 frequency=11>,
 <Record similarity=0.16666666666666666 frequency=18>,
 <Record similarity=0.2 frequency=33>,
 <Record similarity=0.25 frequency=269>,
 <Record similarity=0.2857142857142857 frequency=87>,
 <Record similarity=0.3333333333333333 frequency=81>,
 <Record similarity=0.4 frequency=52>,
 <Record similarity=0.42857142857142855 frequency=566>,
 <Record similarity=0.5 frequency=162>,
 <Record similarity=0.6 frequency=12>,
 <Record similarity=0.6666666666666666 frequency=367>,
 <Record similarity=0.8 frequency=4>,
 <Record similarity=1.0 frequency=254>]


We are calculating the similarity between countries based on their top 5 most produced beer styles. The expected result is a ranking of country pairs based on how similar their beer production preferences are. However, our results show that many country pairs have a similarity score of 1.0, and the similarity only starts to decrease after case 255.

**Why Are There So Many 1.0 Similarity Scores?**

The issue arises because the cosine similarity algorithm used by gds.nodeSimilarity.stream() does not consider the order of elements in the beer style rankings—it only considers which styles appear in the top 5, not their specific rankings or frequencies. If two countries have exactly the same top 5 beer styles, their vectors are identical, and the similarity score is 1.0.

- Many Countries Have the Same Top 5 Beer Styles: Multiple countries share the same top 5 beer styles (the most common styles), so they will all have identical vectors, leading to similarity = 1.0. Since the global beer industry tends to have dominant styles (e.g., American IPA, Pale Ale, Stout), many countries will naturally have the same styles at the top.


In [155]:
# DELETE - Drop graph if it exists
query_drop = """
CALL gds.graph.drop('Country_Beer_Style_Graph', false) YIELD graphName;
"""

try:
    result_drop = execute_read(driver, query_drop)
    pprint(result_drop)
except Exception as e:
    print(e)

[<Record graphName='Country_Beer_Style_Graph'>]


In [156]:
query_store_weights = """
MATCH (c:COUNTRIES)<-[:IN]-(city:CITIES)<-[:IN]-(brewery:BREWERIES)-[:BREWED]->(beer:BEERS)-[:HAS_STYLE]->(s:STYLES)
WITH c, s, COUNT(beer) AS beer_count
ORDER BY c, beer_count DESC  
WITH c, COLLECT(s)[0..5] AS top_styles

UNWIND RANGE(0, SIZE(top_styles)-1) AS idx
WITH c, top_styles[idx] AS style, 5 - idx AS weight

MERGE (c)-[r:TOP_STYLE]->(style)  
SET r.weight = weight;
"""

try:
    result_store_weights = execute_write(driver, query_store_weights)
    pprint(result_store_weights)
except Exception as e:
    print(e)

[]


In [157]:
query_check_weights = """
MATCH (c:COUNTRIES)-[r:TOP_STYLE]->(s:STYLES)
RETURN c.name AS Country, s.name AS Beer_Style, r.weight AS Weight
ORDER BY Country, Weight DESC
LIMIT 20;
"""

try:
    result_weights = execute_read(driver, query_check_weights)
    pprint(result_weights)
except Exception as e:
    print(e)

[<Record Country='AD' Beer_Style='American Blonde Ale' Weight=5>,
 <Record Country='AD' Beer_Style='German Hefeweizen' Weight=4>,
 <Record Country='AD' Beer_Style='English India Pale Ale (IPA)' Weight=3>,
 <Record Country='AD' Beer_Style='Smoke Beer' Weight=2>,
 <Record Country='AD' Beer_Style='German Doppelbock' Weight=1>,
 <Record Country='AE' Beer_Style='Low Alcohol Beer' Weight=5>,
 <Record Country='AE' Beer_Style='German Helles' Weight=4>,
 <Record Country='AG' Beer_Style='American Pale Ale (APA)' Weight=5>,
 <Record Country='AG' Beer_Style='Fruit and Field Beer' Weight=4>,
 <Record Country='AG' Beer_Style='American Porter' Weight=3>,
 <Record Country='AG' Beer_Style='American Light Lager' Weight=2>,
 <Record Country='AG' Beer_Style='American Adjunct Lager' Weight=1>,
 <Record Country='AL' Beer_Style='European Pale Lager' Weight=5>,
 <Record Country='AL' Beer_Style='Bohemian Pilsener' Weight=4>,
 <Record Country='AL' Beer_Style='American Amber / Red Lager' Weight=3>,
 <Record Coun

In [158]:
query_create_graph = """
CALL gds.graph.project(
    'Country_Beer_Style_Graph',
    ['COUNTRIES', 'STYLES'],  
    {
        TOP_STYLE: {
            orientation: 'UNDIRECTED',
            properties: 'weight'
        }
    }
);
"""

try:
    result_create_graph = execute_write(driver, query_create_graph)
    pprint(result_create_graph)
except Exception as e:
    print(e)

[<Record nodeProjection={'COUNTRIES': {'label': 'COUNTRIES', 'properties': {}}, 'STYLES': {'label': 'STYLES', 'properties': {}}} relationshipProjection={'TOP_STYLE': {'orientation': 'UNDIRECTED', 'aggregation': 'DEFAULT', 'type': 'TOP_STYLE', 'properties': {'weight': {'defaultValue': None, 'property': 'weight', 'aggregation': 'DEFAULT'}}}} graphName='Country_Beer_Style_Graph' nodeCount=313 relationshipCount=1626 projectMillis=3053>]


In [154]:
query_check_graph_weights = """
CALL gds.graph.relationshipProperty.stream('Country_Beer_Style_Graph', 'weight')
YIELD sourceNodeId, targetNodeId, propertyValue
RETURN sourceNodeId, targetNodeId, propertyValue
LIMIT 10;
"""

try:
    result_check_graph_weights = execute_read(driver, query_check_graph_weights)
    pprint(result_check_graph_weights)
except Exception as e:
    print(e)

[<Record sourceNodeId=0 targetNodeId=9494220 propertyValue=1.0>,
 <Record sourceNodeId=0 targetNodeId=9494248 propertyValue=3.0>,
 <Record sourceNodeId=0 targetNodeId=9494267 propertyValue=4.0>,
 <Record sourceNodeId=0 targetNodeId=9494284 propertyValue=2.0>,
 <Record sourceNodeId=0 targetNodeId=9494285 propertyValue=5.0>,
 <Record sourceNodeId=1 targetNodeId=9494220 propertyValue=5.0>,
 <Record sourceNodeId=1 targetNodeId=9494228 propertyValue=4.0>,
 <Record sourceNodeId=1 targetNodeId=9494236 propertyValue=3.0>,
 <Record sourceNodeId=1 targetNodeId=9494237 propertyValue=1.0>,
 <Record sourceNodeId=1 targetNodeId=9494241 propertyValue=2.0>]


In [140]:
query_check_weights = """
MATCH (c:COUNTRIES)-[r]->(s:STYLES)
RETURN c.name AS Country, s.name AS Beer_Style, r.weight AS Weight
ORDER BY Country, Weight DESC
LIMIT 20;
"""

try:
    result_weights = execute_read(driver, query_check_weights)
    pprint(result_weights)
except Exception as e:
    print(e)

[<Record Country='AD' Beer_Style='American Blonde Ale' Weight=5>,
 <Record Country='AD' Beer_Style='German Hefeweizen' Weight=4>,
 <Record Country='AD' Beer_Style='English India Pale Ale (IPA)' Weight=3>,
 <Record Country='AD' Beer_Style='Smoke Beer' Weight=2>,
 <Record Country='AD' Beer_Style='German Doppelbock' Weight=1>,
 <Record Country='AE' Beer_Style='Low Alcohol Beer' Weight=5>,
 <Record Country='AE' Beer_Style='German Helles' Weight=4>,
 <Record Country='AG' Beer_Style='American Pale Ale (APA)' Weight=5>,
 <Record Country='AG' Beer_Style='Fruit and Field Beer' Weight=4>,
 <Record Country='AG' Beer_Style='American Porter' Weight=3>,
 <Record Country='AG' Beer_Style='American Light Lager' Weight=2>,
 <Record Country='AG' Beer_Style='American Adjunct Lager' Weight=1>,
 <Record Country='AL' Beer_Style='European Pale Lager' Weight=5>,
 <Record Country='AL' Beer_Style='Bohemian Pilsener' Weight=4>,
 <Record Country='AL' Beer_Style='American Amber / Red Lager' Weight=3>,
 <Record Coun

In [141]:
query_similarity = """
CALL gds.nodeSimilarity.stream('Country_Beer_Style_Graph', {
    similarityMetric: 'JACCARD',
    relationshipWeightProperty: 'weight'  // Use the weight property
})
YIELD node1, node2, similarity

WITH gds.util.asNode(node1) AS Node1, 
     gds.util.asNode(node2) AS Node2, 
     similarity

WHERE labels(Node1) = ['COUNTRIES'] AND labels(Node2) = ['COUNTRIES']  // Only compare countries

RETURN Node1.name AS Country1, 
       Node2.name AS Country2, 
       similarity
ORDER BY similarity DESC
limit 1;
"""

try:
    result_similarity = execute_read(driver, query_similarity)
    pprint(result_similarity)
except Exception as e:
    print(e)


[<Record Country1='CA' Country2='JM' similarity=1.0>]


In [142]:
#Read the similarity reuslts distribution
query = """
    CALL gds.nodeSimilarity.stream('Country_Beer_Style_Graph')
    YIELD similarity
    RETURN similarity, count(*) AS frequency
    ORDER BY similarity DESC;
"""

result_cosine = execute_read(driver, query)
pprint(result_cosine)

[<Record similarity=1.0 frequency=256>,
 <Record similarity=0.8 frequency=4>,
 <Record similarity=0.7317073170731707 frequency=2>,
 <Record similarity=0.6666666666666666 frequency=367>,
 <Record similarity=0.6 frequency=12>,
 <Record similarity=0.5 frequency=168>,
 <Record similarity=0.42857142857142855 frequency=566>,
 <Record similarity=0.4 frequency=52>,
 <Record similarity=0.36 frequency=2>,
 <Record similarity=0.3333333333333333 frequency=97>,
 <Record similarity=0.3103448275862069 frequency=2>,
 <Record similarity=0.3076923076923077 frequency=2>,
 <Record similarity=0.29411764705882354 frequency=4>,
 <Record similarity=0.2857142857142857 frequency=87>,
 <Record similarity=0.2835820895522388 frequency=2>,
 <Record similarity=0.28205128205128205 frequency=4>,
 <Record similarity=0.2753623188405797 frequency=2>,
 <Record similarity=0.2608695652173913 frequency=2>,
 <Record similarity=0.25925925925925924 frequency=2>,
 <Record similarity=0.25 frequency=275>,
 <Record similarity=0.240

We can definaltly see some differences in the similarity score now.

At first, we calculated country similarity based on their top 5 most-produced beer styles, but we noticed that many country pairs had a similarity score of 1.0. This was unexpected since we expected more variation. The issue was that all beer styles in the top 5 were treated equally, without considering their ranking or dominance. Since many countries share the same most popular styles, the algorithm saw them as identical, even when their beer culture might differ.

To fix this, we explicitly assigned weights to the relationships between countries and beer styles, ranking them from 5 to 1 based on their importance. Additionally, we changed the relationship direction from natural to undirected. This allowed beer styles to act as shared connections rather than one-way influences. These changes helped the similarity algorithm better distinguish countries.

#### <a id="most-popular-beer"></a>5.2. Which beer is the most popular considering the number of users who reviewed it? DONE

> first approach using gds.degree.stream(): measures the number of users who reviewed each beer using the weighted degree of connections. This directly answers the question of which beer has the most unique reviewers.

In [143]:
# Step 0 - Clear the graph (if it exists)
try:
    query = """
        CALL gds.graph.drop('most_popular_beer') YIELD graphName;
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

ClientError('Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `most_popular_beer` does not exist on database `neo4j`. It might exist on another database.')


In [144]:
try:
    query = """
        MATCH (u:USERS)-[:POSTED]->(r:REVIEWS)-[:REVIEWED]->(b:BEERS)
        WITH b, COUNT(DISTINCT u) AS user_count
        MERGE (b)-[:REVIEWED_BY {weight: user_count}]->(b);
    """
    result = execute_write(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[]


In [145]:
try:
    query = """
        CALL gds.graph.project(
            'most_popular_beer',
            ['BEERS'],
            {
                REVIEWED_BY: {
                    orientation: 'UNDIRECTED',
                    properties: 'weight'
                }
            }
        );

    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record nodeProjection={'BEERS': {'label': 'BEERS', 'properties': {}}} relationshipProjection={'REVIEWED_BY': {'orientation': 'UNDIRECTED', 'aggregation': 'DEFAULT', 'type': 'REVIEWED_BY', 'properties': {'weight': {'defaultValue': None, 'property': 'weight', 'aggregation': 'DEFAULT'}}}} graphName='most_popular_beer' nodeCount=358752 relationshipCount=378732 projectMillis=3392>]


In [146]:
try:
    query = """
        CALL gds.degree.stream('most_popular_beer', {relationshipWeightProperty: 'weight'})
        YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).name AS beer_name, score AS num_users
        ORDER BY num_users DESC
        LIMIT 10;

    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record beer_name='Breakfast Stout' num_users=9656.0>,
 <Record beer_name='90 Minute IPA' num_users=8788.0>,
 <Record beer_name='Pliny The Elder' num_users=8426.0>,
 <Record beer_name='Two Hearted Ale' num_users=8336.0>,
 <Record beer_name='Heady Topper' num_users=7946.0>,
 <Record beer_name='Sculpin IPA' num_users=7918.0>,
 <Record beer_name='Hopslam Ale' num_users=7686.0>,
 <Record beer_name='KBS (Kentucky Breakfast Stout)' num_users=7418.0>,
 <Record beer_name='Bourbon County Brand Stout' num_users=7230.0>,
 <Record beer_name='Old Rasputin' num_users=7058.0>]


> second approach using gds.pageRank.stream(): which measures the importance of a beer in the network based on review connections.

In [147]:
try:
    query = """
        CALL gds.graph.project.cypher(
            'most_popular_beer',
            'MATCH (u:USERS) RETURN id(u) AS id
             UNION
             MATCH (b:BEERS) RETURN id(b) AS id',
            'MATCH (u:USERS)-[:POSTED]->(r:REVIEWS)-[:REVIEWED]->(b:BEERS)
             WITH id(u) AS source, id(b) AS target, COUNT(DISTINCT u) AS weight
             RETURN source, target, weight'
        );
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

ClientError("Failed to invoke procedure `gds.graph.project.cypher`: Caused by: java.lang.IllegalArgumentException: A graph with name 'most_popular_beer' already exists.")


In [148]:
try:
    query = """
        CALL gds.pageRank.stream('most_popular_beer', {relationshipWeightProperty: 'weight'})
        YIELD nodeId, score
        RETURN gds.util.asNode(nodeId).name AS beer_name, score AS popularity
        ORDER BY popularity DESC
        LIMIT 10;
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record beer_name='Café Negro Coffee Infused Porter' popularity=0.9612404689154858>,
 <Record beer_name='Lectio Divina' popularity=0.9612404689154858>,
 <Record beer_name='Winter Warmer' popularity=0.9612404689154858>,
 <Record beer_name="Four C's American Pale Ale" popularity=0.9612404689154858>,
 <Record beer_name='Yvan The Great: Belgian-Style Blonde (Beer Camp Across America)' popularity=0.9612404689154858>,
 <Record beer_name='Red Rock Elephino' popularity=0.9612404689154858>,
 <Record beer_name="Tart 'n Juicy Sour IPA" popularity=0.9612404689154858>,
 <Record beer_name='Cascade Sang Rouge' popularity=0.9612404689154858>,
 <Record beer_name='Cuivre' popularity=0.9612404689154858>,
 <Record beer_name="Pile O' Dirt Porter" popularity=0.9612404689154858>]


`Chosen Approach: Degree Centrality`

- The first approach, is the most correct because it directly measures how many unique users have reviewed each beer. Since the problem 2 asks for the most popular beer based on the number of users who reviewed it, degree centrality (counting unique connections) is the best fit.
- Using gds.degree.stream(), we calculate the weighted degree of each beer, which directly reflects the number of unique users who reviewed it. This method is more accurate for the given problem because it counts unique user interactions rather than measuring network influence like PageRank.

#### <a id="user-communities"></a>5.3. How many communities are formed from these relationships? How many users are in the three largest communities? OPTIONAL DUE TO MEMORY ISSUES

The reason we not choose this as one of the question directly is because we keep running into memory problems.

In [149]:
#Step 0 - Clear the graph (if it exists)
try:
    query = """
        CALL gds.graph.drop('beer_communities') YIELD graphName;
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

ClientError('Failed to invoke procedure `gds.graph.drop`: Caused by: java.util.NoSuchElementException: Graph with name `beer_communities` does not exist on database `neo4j`. It might exist on another database.')


In [150]:
# Step 1 - Create the Graph Projection
try:
    query = """
        MATCH (u1:USERS)-[:POSTED]->(r1:REVIEWS)-[:REVIEWED]->(b:BEERS)<-[:REVIEWED]-(r2:REVIEWS)<-[:POSTED]-(u2:USERS)
        WHERE u1 <> u2
        WITH u1, u2, COUNT(DISTINCT b) AS weight
        LIMIT 100000  // Process only 100,000 relationships
        MERGE (u1)-[r:REVIEWED_SAME_BEER {weight: weight}]->(u2);
    """
    result = execute_write(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[]


We connect users who reviewed the same beer and calculate a weight based on how many distinct beers both users reviewed. To prevent excessive memory usage, we limit the processing to 100,000 relationships. Unfortunately, due to memory constraints, we were unable to test this approach with a larger dataset. This means the solution is not fully complete, as it does not analyze all possible relationships. However, we decided to keep the logic of the code:

In [151]:
try:
    query = """
        MATCH (u1:USERS)-[r:REVIEWED_SAME_BEER]->(u2:USERS)
        RETURN count(r) AS total_relationships;
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record total_relationships=100000>]


In [159]:
try:
    query = """
        MATCH (u1:USERS)-[r:REVIEWED_SAME_BEER]->(u2:USERS)
        RETURN u1.name AS user1, u2.name AS user2, r.weight AS weight
        LIMIT 10;
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record user1='bluejacket74' user2='GratefulBeerGuy' weight=34>,
 <Record user1='_dirty_' user2='GratefulBeerGuy' weight=1>,
 <Record user1='_dirty_' user2='CJDUBYA' weight=1>,
 <Record user1='CJDUBYA' user2='GratefulBeerGuy' weight=1>,
 <Record user1='CJDUBYA' user2='_dirty_' weight=1>,
 <Record user1='GratefulBeerGuy' user2='_dirty_' weight=1>,
 <Record user1='GratefulBeerGuy' user2='bluejacket74' weight=34>,
 <Record user1='GratefulBeerGuy' user2='CJDUBYA' weight=1>,
 <Record user1='LukeGude' user2='GratefulBeerGuy' weight=24>,
 <Record user1='LukeGude' user2='_dirty_' weight=2>]


In [160]:
try:
    query = """
        CALL gds.graph.project.cypher(
            'beer_communities',
            'MATCH (u:USERS) RETURN id(u) AS id',
            'MATCH (u1:USERS)-[r:REVIEWED_SAME_BEER]->(u2:USERS)
             WHERE r.weight > 1
             RETURN id(u1) AS source, id(u2) AS target, r.weight AS weight'
        );
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record nodeQuery='MATCH (u:USERS) RETURN id(u) AS id' relationshipQuery='MATCH (u1:USERS)-[r:REVIEWED_SAME_BEER]->(u2:USERS)\n             WHERE r.weight > 1\n             RETURN id(u1) AS source, id(u2) AS target, r.weight AS weight' graphName='beer_communities' nodeCount=106645 relationshipCount=50442 projectMillis=452>]


In [161]:
#Step 2 - Get the Number of Communities
try:
    query = """
        CALL gds.labelPropagation.stats('beer_communities')
        YIELD communityCount, ranIterations, didConverge;
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record communityCount=76786 ranIterations=3 didConverge=True>]


In [162]:
#Step 3 - Find the Three Largest Communities
try:
    query = """
        CALL gds.labelPropagation.stream('beer_communities')
        YIELD nodeId, communityId AS Community
        WITH Community, count(*) AS freq
        ORDER BY freq DESC
        LIMIT 3
        RETURN Community, freq;
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record Community=9494329 freq=28854>,
 <Record Community=9494327 freq=1005>,
 <Record Community=9494328 freq=3>]


In [163]:
#Step 4 - List Some Users in a Specific Community
try:
    query = """
        CALL gds.labelPropagation.stream('beer_communities')
        YIELD nodeId, communityId AS Community
        WITH gds.util.asNode(nodeId).name AS Name, Community
        WHERE Community = 9494327
        RETURN Name
        LIMIT 20;
    """
    result = execute_read(driver, query)
    pprint(result)
except Exception as e:
    pprint(e)

[<Record Name='_dirty_'>,
 <Record Name='MBruckerATC'>,
 <Record Name='PDOR1960'>,
 <Record Name='goblue30'>,
 <Record Name='Lcoblentz'>,
 <Record Name='Jakethesnake503'>,
 <Record Name='Loogs'>,
 <Record Name='Schmaltz33'>,
 <Record Name='prtwin2a'>,
 <Record Name='arr3388'>,
 <Record Name='Sloanerb'>,
 <Record Name='Dloschiavo'>,
 <Record Name='mmenard169'>,
 <Record Name='JonB25'>,
 <Record Name='bibo_ergo_sum'>,
 <Record Name='digboy'>,
 <Record Name='DrewGingras'>,
 <Record Name='Glennp64'>,
 <Record Name='AndrewCataldo23'>,
 <Record Name='yossle'>]
